In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


[MYCODE] .env 파일 로드 
- opeanAIKey Load
- 서버에 환경셋팅

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever()

user_msg = "What is Task Decomposition?"
retrieved_docs = retriever.invoke(user_msg)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke(
    {"context": format_docs(retrieved_docs), "question": user_msg}
)
print(user_prompt)

In [ ]:
response = llm.invoke(user_prompt)
print(response.content)